In [1]:
%pylab inline
import pandas as pd
import os.path
from pandas.errors import EmptyDataError
import gffpandas.gffpandas as gffpd


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
Gene_ids = {'NLBDLAKK':'Pch','PBMDCMNA':'Pch', 'MAOEKBOG':'Ab', 'BDBMPBCG':'Ab',
           'BAJJDFMB':'Ea','AEIEMNPP':'Ea', 'EAHMIHHM':'Pa','MOJAEBOL':'Pa', 
           'MOFKNFJH':'Pf', 'HLOAJDCL':'Pf', 'JFAGEJMB':'Sm'}


def _split_atts(atts):
    """Split a feature string into attributes."""
    splits_list = [a.split("=") for a in atts.split(";") if "=" in a]
    return {l[0]: "=".join(l[1:]) for l in splits_list}



def con_gene_names(name, gid = Gene_ids):
#     print(name)
    for key in gid.keys():
        name = name.replace(key, gid[key])
    return name


def taking_downstream_genes(row):
    
#     print(row.name)
    if str(row['gene_position']).startswith("intergenic"):
        splitted = str(row['gene_position']).split("/")
        genes = row['gene_name'].split('/')
        
        for i in range(len(splitted)):
            far_from_gene = re.findall(r'\d+', splitted[i])[0]
            if "-" not in splitted[i] or int(far_from_gene) > 150:
                genes[i] = None
                                
        L_name = [x for x in genes if x != None]
        if len(L_name) == 1:
            L_name = L_name[0]
        row['eff_gene'] = None if not L_name else L_name

        if (len(str(row['gene_name']).split(','))==1) & ('[' in str(row['gene_name'])):
            row['eff_gene'] = row['gene_name'][0]
            
    elif (row['mutation_category']=='large_deletion')&(type(row['gene_product'])==list):
        row['eff_gene'] = row['gene_product']
        
    else:
        row['eff_gene'] = row['gene_name']

    return row


def intergenic_prod(row):
    
#     print(row.name)
    if str(row['gene_position']).startswith("intergenic"):
#         print(row['strain'])
        splitted = row['gene_position'].replace('intergenic ', '').replace('(', '').replace(')', '').split('/')
        
        if type(row['gene_product']) == list:
            
            gen_prods = row['gene_product']
            
        else:    
            gen_prods = row['gene_product'].split('/')
            
#         print(row.name, splitted)
        if splitted == ['–', '–']:
            L_prods = np.nan
        else:
            for i in range(len(splitted)):
        #             dist_from_gene = re.findall(r'\d+', splitted[i])[0]
                if abs(int(splitted[i])) > 150:
                    gen_prods[i] = None
            gen_prods.append(row['gene_product'])        
            L_prod = [x for x in gen_prods if x != None]
        #         print(L_prod)

            row['gene_product'] = row['gene_name'] if not L_prod else L_prod

            if (len(str(row['gene_name']).split(','))==1) & ('[' in str(row['gene_name'])):
                row['gene_name'] = row['gene_name'][0]
            if (len(str(row['gene_product']).split(','))==1) & ('[' in str(row['gene_product'])):
                row['gene_product'] = row['gene_product'][0]

    elif '[' in str(row['gene_name']):
        row['gene_name'] = row['gene_name'].replace('[', '').replace(']', '')
    return row


def create_mutations_database(mapping_file, ser_num = 'serial number'):
    
    '''Unify the tsv tables from varios strains
    Adapted from @galtol'''
    
    missing = []
    zero_mut = []
    header = ['Species', 'ev-comm', 'Pre-Na','ev_ident', "strain", "type", "snp_type", "mutation_category",
                "gene_name", "gene_product", "gene_position",
                "position_end", "position_start", "genes_inactivated",
                "size",  "ref_seq", "new_seq",
                'repeat_length', 'repeat_new_copies', 'repeat_ref_copies',
                'repeat_seq']
    combined = pd.DataFrame(columns=header)
    
    for i in mapping_file.index:
        num = mapping_file.loc[i, ser_num]
        
        folder = mapping_file.loc[i, 'Pre-Na']
        if not os.path.exists(f'all_tsvs/{folder}/{num}.tsv'):
            print(f"no file {num}")
            missing.append(i)
            continue
        try:
            data = pd.read_csv(f'all_tsvs/{folder}/{num}.tsv', sep="\t")
            sub_header = [value for value in list(data.columns) if value in header]
            sub = data[sub_header]
            sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
            sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
            sub["Species"] = mapping_file.loc[i, 'Species']
            sub["ev-comm"] = mapping_file.loc[i, 'ev-comm']
            sub["Pre-Na"] = mapping_file.loc[i, 'Pre-Na']
            
            
            combined = combined.append(sub)
            
        except EmptyDataError:
            print(f"zero mutations at {num}")
            zero_mut.append(i)
            continue
        
    combined = combined.reset_index(drop=True)
    combined['intergenic'] = combined.gene_position.astype(str).apply(lambda x: True if x.startswith("intergenic") else False)
    combined['freamshift'] = combined['size'].fillna(0) % 3 != 0
    return combined, missing, zero_mut


def rearange_genes(genes):
    
    if genes == None:
        return None
    elif type(genes) == list:
        return [gene.replace('[', '').replace(']', '') for gene in genes]
    else:
        li = [gene.replace('[', '').replace(']', '') for gene in genes.split(',')]
        if len(li) == 1:
            return li[0]
        else:
            return li





In [13]:
old_cols = ['start', 'end', 'strand', 'Name', 'locus_tag', 'product']
new_cols = ['start', 'end', 'type', 'conf', 'em_COG_cat', 'em_PFAMs', 'em_Preferred_name', 'em_desc', 'gc_cont']


for sp in ['Ea', 'Pa', 'Pch', 'Sm', 'Pf', 'Ab']:
#     print(sp)
    sumd = gffpd.read_gff3(f'Ancestors/{sp}.gff')
    sumd.df = sumd.df.dropna()
    CR = sumd.attributes_to_columns()
    CR['fraction_of_genome'] = (CR['end']-CR['start'])/CR['end'].max()
    CR = sumd.attributes_to_columns()
    CR.loc[CR['locus_tag'].isna(), ['ID', 'Name', 'gene', 'locus_tag']] = 'CRISPER_Seq' 
    CR['locus_tag'] = CR['locus_tag'].apply(lambda x: con_gene_names(x))
    
    
    sumd = gffpd.read_gff3(f'Ancestors/out_{sp}.emapper.decorated.gff')
    sumd.df = sumd.df.dropna()
    temp = sumd.df.attributes.apply(lambda row: _split_atts(row))
    emap = sumd.df.join(temp.apply(pd.Series), lsuffix='_left', rsuffix='_right')
    
    pd.merge(CR[old_cols], emap[new_cols], on=['start', 'end'], how = 'left').to_excel(f'Ancestors/Sum_data/{sp}.xlsx')
    

C:\Users\nitta\Anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',
C:\Users\nitta\Anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',
C:\Users\nitta\Anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',
C:\Users\nitta\Anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',
C:\Users\nitta\Anaconda3\lib\site-packages\gffpandas\gffpand

In [16]:
GENE_NAME_CONVERTOR = {'lutR': 'IV02_29395', 'rspR': 'ydfH', 'ansP2': 'ansP', 'lpxP': 'htrB', 'acs': 'acsA', 'EAHMIHHM': 'rssA', 'capB': 'capA', 'proY': 'ytnA', 'NLBDLAKK': 'pssA', 'norR': 'fleQ', 'ispU': 'uppS',
                       'hemR': 'phuR', 'potA': 'potA5', 'uvrY': 'uhpA', 'copA': 'ccoI', 'spuE': 'potF2', 'zraR': 'cbrB', 'rarD': 'rarD-2', 'spuD': 'potF1', 'MOFKNFJH': 'algZ', 'dht': 'hydA'}

Gene_ids = {'NLBDLAKK':'Pch','PBMDCMNA':'Pch', 'MAOEKBOG':'Ab', 'BDBMPBCG':'Ab',
           'BAJJDFMB':'Ea','AEIEMNPP':'Ea', 'EAHMIHHM':'Pa','MOJAEBOL':'Pa', 
           'MOFKNFJH':'Pf', 'HLOAJDCL':'Pf', 'JFAGEJMB':'Sm'}


strains = pd.read_csv('strains.csv')

strains['Species'] = strains['Species'].apply(lambda x:x.replace(" ", ""))
strains = strains[~strains['serial number'].isna()]

mut_table, missing, zero_mut = create_mutations_database(strains)
strains = strains.loc[~strains.index.isin(missing)]

C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

zero mutations at 21
zero mutations at 64


C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

zero mutations at 121121_225
no file 021021_54
no file 17
no file 2


...

C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

zero mutations at 80
zero mutations at 88


C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["ev_ident"] = mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub["strain"] = mapping_file.loc[i, 'Species']+ '_' +mapping_file.loc[i, 'ev-ident']
C:\Users\nitta\AppData\Local\Temp\ipykernel_26888\1710113099.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [17]:
# SNP size is 0
mut_table.loc[(mut_table['type']=='SNP')&(mut_table['size'].isna()), 'size'] = 0
# Find the size of indels
mut_table.loc[(mut_table['mutation_category']=='small_indel')&(mut_table['size'].isna()), 'size'] = mut_table.loc[(mut_table['mutation_category']=='small_indel')&(mut_table['size'].isna()), 'new_seq'].apply(lambda x:len(x))
# In lare deletions that hit mutliple genes position is nan, change to poly to make it more informative 
mut_table.loc[mut_table.gene_position.isna(), 'gene_position'] = 'Poly'
# Hypo proteins are assined a random sequnce of letters change to species is name to make it comparable between naive and pre-adapted
mut_table['gene_name'] = mut_table['gene_name'].apply(lambda x: con_gene_names(x))
mut_table['gene_product'] = mut_table['gene_product'].apply(lambda x: con_gene_names(x))

# find which exact sequnce mutations apear more then once, 
mut_freq = mut_table.groupby(['Species', 'gene_name', 'gene_position', 'size']).count()['strain']
# Save to file and inspect mannualy
mut_freq[mut_freq>1].to_excel('Manu_non_anotated.xlsx')
strains_before = mut_table['strain'].unique()


In [18]:
# Mannuall inspected
manual = pd.read_excel('Manu.xlsx', index_col=[0, 1, 2, 3])
manual = manual[manual['verdict']=='Remove']
mut_table = mut_table[~mut_table['strain'].isin(['Pch_anc', 'Sm_anc'])] # These are ancestors, remove from this table
strains = strains[strains['ev-comm']!='anc']

mut_all = mut_table.copy() # Mut_all includes all mutations with none filtered
# Filter out: 
remove_dup = mut_table[(mut_table['gene_name'].isin(manual.reset_index()['gene_name']))&(mut_table['gene_position'].astype(str).isin(manual.reset_index()['gene_position'].astype(str)))]
mut_table = mut_table.drop(remove_dup.index, axis =0)


### Unique ID for each strain
strains['strain'] = strains['Species']+'_'+strains['ev-ident']
### Number of mutations for each strain
n_mut = mut_table.groupby('strain').agg({'gene_name':'count'})

### in Pf_Exp26-p3D10, only ~70% of the reads were mapped, and a very high number of mutations - probably a contamination
mut_table = mut_table[mut_table['strain']!= 'Pf_Exp26-p3D10']
strains = strains[strains['strain']!= 'Pf_Exp26-p3D10']

## Update such that mutations that were depricated are removed from cont
zero_mut = zero_mut + list(strains[strains['strain'].isin(strains_before[~np.in1d(strains_before,mut_table['strain'].unique())])].index)
strains['n_mut'] = strains.apply(lambda x:n_mut.loc[x.strain] if x.name not in zero_mut else 0, axis = 1)
strains = strains[~strains['anc']]
#Easier annotation for treatments
strains['m-c'] = strains.apply(lambda x:'mono' if x['Species']==x['ev-comm'] else 'co', axis = 1)
strains.to_excel('strains_with_num.xlsx')

# 150 bp from gene, count as affected
mut_table = mut_table.apply(lambda x:intergenic_prod(x), axis = 1)
# Give unique mutations per id
mut_table['mutation_id'] = arange(len(mut_table))
mut_table['gene_product'] = mut_table['gene_product'].apply(rearange_genes)
mut_table['eff_gene'] = np.nan
mut_table = mut_table.apply(lambda x:taking_downstream_genes(x), axis = 1)

mut_table.loc[mut_table['gene_product']=='hypothetical protein', 'gene_product'] = mut_table.loc[mut_table['gene_product']=='hypothetical protein', 'gene_name']
mut_table.to_excel('num_mut.xlsx')

In [19]:
strains['strain'] = strains['Species']+'_'+strains['ev-ident']
n_mut = mut_all.groupby('strain').agg({'gene_name':'count'})

mut_all = mut_all[mut_all['strain']!= 'Pf_Exp26-p3D10']
mut_all = mut_all[mut_all['gene_position']!='intergenic (–/–)']
mut_all = mut_all.apply(lambda x:intergenic_prod(x), axis = 1)
mut_all['mutation_id'] = arange(len(mut_all))
mut_all['gene_product'] = mut_all['gene_product'].apply(rearange_genes)
mut_all['eff_gene'] = np.nan
mut_all = mut_all.apply(lambda x:taking_downstream_genes(x), axis = 1)

mut_all.loc[mut_all['gene_product']=='hypothetical protein', 'gene_product'] = mut_all.loc[mut_all['gene_product']=='hypothetical protein', 'gene_name']
mut_all.to_excel('mut_all.xlsx')

